In [80]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import shutil

from zipfile import ZipFile, Path
from nltk.corpus import stopwords
from cleantext import clean


In [81]:
df = pd.read_csv('../datasets/songs.zip')
df.head()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.01170,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en


In [82]:
df = df.drop(['track_popularity', 'track_album_release_date'], axis='columns')
df.head()

,track_id,track_name,track_artist,lyrics,track_album_id,track_album_name,playlist_name,playlist_id,playlist_genre,playlist_subgenre,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,1srJQ0njEQgd8w4XSqI4JQ,Trip,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,rock,classic rock,...,-10.068,1,0.0236,0.27900,0.01170,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",3z04Lb9Dsilqw68SHt6jLB,Love & Loss,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,rock,hard rock,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",6oZ6brjB8x3GoeSYdwJdPc,Gold,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,r&b,new jack swing,...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,Christmas Soul,6FZYc2BvF7tColxO8PBShV,r&b,neo soul,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,pop,dance pop,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en


In [83]:
print(df.columns)
print(df['playlist_genre'].value_counts())
print(df['playlist_subgenre'].value_counts())

Index(['track_id', 'track_name', 'track_artist', 'lyrics', 'track_album_id',
       'track_album_name', 'playlist_name', 'playlist_id', 'playlist_genre',
       'playlist_subgenre', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'language'],
      dtype='object')
pop      3993
rock     3521
rap      3391
r&b      3326
latin    2178
edm      2045
Name: playlist_genre, dtype: int64
indie poptimism              1288
southern hip hop             1158
neo soul                     1001
electropop                    964
urban contemporary            936
hard rock                     926
classic rock                  924
post-teen pop                 891
new jack swing                881
gangster rap                  865
dance pop                     850
permanent wave                843
album rock                    828
hip hop                       689
trap                   

Removing all the songs that are not in english.

In [84]:
df = df[df['language'].str.contains('en') == True]
df.head()

,track_id,track_name,track_artist,lyrics,track_album_id,track_album_name,playlist_name,playlist_id,playlist_genre,playlist_subgenre,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",3z04Lb9Dsilqw68SHt6jLB,Love & Loss,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,rock,hard rock,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",6oZ6brjB8x3GoeSYdwJdPc,Gold,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,r&b,new jack swing,...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,Christmas Soul,6FZYc2BvF7tColxO8PBShV,r&b,neo soul,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,pop,dance pop,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en
5,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,"Hold your breath, don't look down, keep trying...",3GNzXsFbzdwM0WKCZtgeNP,Chapters,urban contemporary,4WiB26kw0INKwbzfb5M6Tv,r&b,urban contemporary,...,-6.157,1,0.0550,0.28000,0.00000,0.0975,0.305,147.764,224720,en


In [85]:
df['language'].value_counts()

en    15405
Name: language, dtype: int64

In [86]:
def clean_text(text):
    text = text.lower()
    text = text.encode('ascii', 'ignore').decode()
    stop_words = stopwords.words('english')
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub(r'\w*\d+\w*', '', text)
    text = re.sub('\s{2,}', ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub(' s ', ' ', text)
    text = re.sub(' m ', ' ', text)
    text = clean(text, no_emoji=True)
    return text

In [87]:
df['lyrics'] = df.lyrics.apply(clean_text)
df['playlist_name'] = df.playlist_name.apply(clean_text)

In [88]:
df['playlist_name'].value_counts()

indie poptimism       544
permanent wave        404
urban contemporary    344
electropop            336
post teen pop         284
                     ... 
latin pop               1
academia reggaetn       1
trapperz brasil         1
is omar                 1
trap land               1
Name: playlist_name, Length: 353, dtype: int64

Dropping any null values. (There are none in this specific dataset, but it's standard practice.)

In [89]:
print(df.shape)
df = df.dropna()
df.shape

(15405, 23)


(15405, 23)

Rounding Decimal Values. A lot of the metrics about the audio itself had very large floating decimal values (some were even 10^-9!) so I've rounded to an appropriate decimal value for each column where it was needed.

In [95]:
df['danceability'].astype(float).round(decimals=3)
df['energy'].astype(float).round(decimals=4)
df['loudness'].astype(float).round(decimals=4)
df['speechiness'].astype(float).round(decimals=4)
df['acousticness'].round(decimals=5)
df['instrumentalness'].round(decimals=5)
df['liveness'].round(decimals=5)
df['valence'].round(decimals=4)
df['tempo'].round(decimals=3)
df['instrumentalness']


<bound method IndexOpsMixin.value_counts of 1        0.009940
2        0.007230
3        0.000000
4        0.000000
5        0.000000
           ...   
18448    0.000000
18449    0.000000
18450    0.000006
18451    0.000428
18452    0.000000
Name: instrumentalness, Length: 15405, dtype: float64>

Writing the cleaned data to a zip file. Which is archived as a csv file.

In [91]:
df.to_csv('cleaned_RNN_dataset.zip', compression={'method': 'zip', 'archive_name': 'cleaned_RNN_dataset.csv'})
shutil.move('cleaned_RNN_dataset.zip', '../datasets/cleaned_RNN_dataset.zip')

'../datasets/cleaned_RNN_dataset.zip'